In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render ul li, div.text_cell_render ol li p, code{font-size:22pt; line-height:30px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

# 벡터DB : Chroma vs. Pinecorn
- Chroma : 인메모리 vector DB, 로컬 vector DB
- Pineconr : 클라우드 vector DB
-      (htt[s://www

# 0. 패키지 설치

In [2]:
%pip install -q pinecone langchain-pinecone --no-warn-script-location

Note: you may need to restart the kernel to use updated packages.


# 1. knowledge Base 구성을 위한 데이터 생성

In [3]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = Docx2txtLoader('data/소득세법(법률)(제21065호)(20260102).docx')
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, 
    chunk_overlap=200,
    # separators=["\n\n", "\n", " ", ""]
)
document_list = loader.load_and_split(text_splitter=text_splitter)
len(document_list)



193

In [4]:
# embedding : OpenAI API text-embedding-3-large
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
load_dotenv()
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

In [5]:

%%time
# pinecone vector database
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
import os
pc = Pinecone(
    api_key=os.getenv("PINECONE_API_KEY")
)
# 데이터를 처음 업로드할 때
index_name = "tax-index"
database = PineconeVectorStore.from_documents(
    documents=document_list,
    embedding=embedding,
    index_name=index_name
)


C:\Users\Admin\anaconda3\envs\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CPU times: total: 5.12 s
Wall time: 20.6 s


2. 답변 생성을 위한 Retrieval

In [9]:
query = "연봉이 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_docs = database.similarity_search(query, k=3) # 기본k값:4

In [10]:
# retrieved_docs[2].page_content
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

In [11]:
retriever = database.as_retriever(
    search_kwargs={"k":3}
)
retrived_docs = retriever.invoke(query)
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

# 3. 답변 생성

In [13]:

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = "gpt-4.1-nano")

In [14]:
# upstage에서 받은 20$로 llm을 사용하고 싶다면
from langchain_upstage import ChatUpstage
llm = ChatUpstage(
    model = "solar-pro2",
    reasoning_effort="high" #느리지만 더 깊게 추론함 (low, medium)
)

In [15]:
prompt = f"""[identity]
- 당신은 최고의 한국 소득세법 전문가입니다
- [context]를 참고해서 사용자의 질문에 답변해 주세요.
- [context]는 다음과 같아요
{retrieved_doc}
- 질문 : {query}"""

In [16]:

ai_message = llm.invoke(prompt)

In [17]:
ai_message

AIMessage(content='연봉 5천만 원인 직장인의 소득세 계산은 다음과 같은 단계를 거칩니다. 단, 제시된 [context]에는 소득공제율(소득세법 제47조) 및 세율(소득세법 제55조) 관련 구체적 내용이 누락되어 있어 일반적인 한국 소득세법 체계를 기반으로 설명드립니다.\n\n---\n\n### **1. 근로소득 금액 산정**\n- **총급여액**: 연봉 5,000만 원 (비과세 소득 제외)\n- **근로소득공제**: 총급여액에 따라 공제율이 적용됩니다.  \n  (예시: 2023년 기준)\n  - 1,500만 원 이하: 70% 공제  \n  - 1,500만~4,500만 원: 40% 공제  \n  - 4,500만~1억 원: 15% 공제  \n  - 1억 원 초과: 10% 공제  \n\n  **계산**:  \n  - 1,500만 원 × 70% = 1,050만 원  \n  - (4,500만 원 - 1,500만 원) × 40% = 1,200만 원  \n  - (5,000만 원 - 4,500만 원) × 15% = 75만 원  \n  **총 공제액**: 1,050 + 1,200 + 75 = **2,325만 원**  \n  **근로소득금액**: 5,000만 원 - 2,325만 원 = **2,675만 원**\n\n---\n\n### **2. 과세표준 및 세율 적용**\n- **과세표준**: 근로소득금액 2,675만 원 (기타 소득·공제 없음 가정)  \n- **종합소득세율** (2023년 기준 누진세율):  \n  | 과세표준 구간 | 세율 | 누진공제 |  \n  |--------------|------|----------|  \n  | ~1,200만 원 | 6% | - |  \n  | 1,200~4,600만 원 | 15% | 108만 원 |  \n  | 4,600~8,800만 원 | 24% | 522만 원 |  \n\n  **계산**:  \n  - 1,200만 원 × 6% = 72만 원  \n  - (2,675만 원 - 1,200만 원) × 15% = 2